In [ ]:
df = clean_up_mongo_coll(db.main_redo)

user_song = turn_df_to_util_mat(
    df, 'new_song_producer', 'sampled_artist_song')

user_inds, item_inds = get_indices_of_test_set_values(user_song, 5)

train, test = make_train_set_and_test_set(user_inds, item_inds, user_song)

In [ ]:
# Train model for producer- sampled song utility matrix

# filter so at least 6 samples per producer and at least 1 sample per artist.
train_lim, test_lim, user_lim, item_lim = filter_dataset_by_requisite_interactions(
        train, test, 5, 5)

rank_score, pop_rank_score = get_rank_and_pop_score_from_train_test_model(
    train_lim, test_lim, user_lim, item_lim, 20, 30, 45)

print(
"Model rank score: {} \n\
Popularity rank score: {}".format(rank_score, pop_rank_score))

pd.DataFrame({"Model Rank Score": [round(rank_score, 3)], 
                     "Popularity Rank Score" : [round(pop_rank_score, 3)]})

In [ ]:

model = implicit.als.AlternatingLeastSquares(
        factors=best_factors, iterations=best_iters, regularization= best_lambda)

# train the model on a sparse matrix of item/user/confidence weights
sparse_user_song_filt = csr_matrix(train_lim)
model.fit(sparse_user_song_filt)

# calculate predictions

predictions_user_song = model.item_factors.dot(model.user_factors.T)